# In this Notebook an analysis of the **substitutions impact on the game will be explored**, based solely on the team game statistics and the player ratings

This work was applied to Manchester City 2017/2018 season

In [238]:
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt

from sklearn.svm import SVC

import unidecode

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import requests

# Load data

Load event data from Wyscout source

**Events**

In [65]:
#data from https://figshare.com/collections/Soccer_match_event_dataset/4415000/2
with open('data/events_England.json') as f:
    data = json.load(f)
    

df_events = pd.DataFrame(data)
df_events.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175


In [66]:
event_ids_mapping = {i: rows['eventName'].unique()[0] for i, rows in df_events.groupby(['eventId'])}
subevent_ids_mapping = {i: rows['subEventName'].unique()[0] for i, rows in df_events.groupby(['subEventId'])}

In [67]:
#different types of events

event_ids_mapping

{1: 'Duel',
 2: 'Foul',
 3: 'Free Kick',
 4: 'Goalkeeper leaving line',
 5: 'Interruption',
 6: 'Offside',
 7: 'Others on the ball',
 8: 'Pass',
 9: 'Save attempt',
 10: 'Shot'}

In [68]:
#different types of subevents

subevent_ids_mapping

{10: 'Air duel',
 11: 'Ground attacking duel',
 12: 'Ground defending duel',
 13: 'Ground loose ball duel',
 20: 'Foul',
 21: 'Hand foul',
 22: 'Late card foul',
 23: 'Out of game foul',
 24: 'Protest',
 25: 'Simulation',
 26: 'Time lost foul',
 27: 'Violent Foul',
 30: 'Corner',
 31: 'Free Kick',
 32: 'Free kick cross',
 33: 'Free kick shot',
 34: 'Goal kick',
 35: 'Penalty',
 36: 'Throw in',
 40: 'Goalkeeper leaving line',
 50: 'Ball out of the field',
 51: 'Whistle',
 70: 'Acceleration',
 71: 'Clearance',
 72: 'Touch',
 80: 'Cross',
 81: 'Hand pass',
 82: 'Head pass',
 83: 'High pass',
 84: 'Launch',
 85: 'Simple pass',
 86: 'Smart pass',
 90: 'Reflexes',
 91: 'Save attempt',
 100: 'Shot',
 '': ''}

In [ ]:
# get Man City ID

with open('data/teams.json') as f:
    teams_data = json.load(f)
teams_df = pd.DataFrame(teams_data)

man_city_id = str(teams_df[teams_df['officialName'] == 'Manchester City FC'].iloc[0]['wyId'])
man_city_id

**Players**

In [469]:
with open('data/players.json') as f:
    players_data = json.load(f)

players_df = pd.DataFrame(players_data)
players_df['role'][1]

{'code2': 'DF', 'code3': 'DEF', 'name': 'Defender'}

**Matches**

In [70]:
# fetch substitutions

with open('data/matches_England.json') as f:
    matches_data = json.load(f)

matches_df = pd.DataFrame(matches_data)

In [71]:
city_games_df = matches_df[matches_df.apply(lambda x: str(man_city_id) in x['teamsData'].keys(), axis=1)]

## Get Stats

In this case there are two important data sources that will be used:
- Game stats, until the moment of the substitutions
- Final rating of the player that entered in the game

### 1. Game stats, until the moment of the substitutions

These stats will be extracted from the event data (using events_id, sub_events_id and tags_id), specifically the extracted stats are:
- Number of shots
- Number of shots that resulted in goal
- Number of shots that resulted in an opportunity
- Number of passes
- Number of key passes
- Number of cross passes
- Number of assists
- Number of passes that resulted in an opportunity
- Number of Free kicks
- Number of corners
- Number of player accelerations
- Number of shots conceded
- Number of yellow cards to the opposition team
- Number of red cars to the opposition team
- Number of saves of the opposition team
- Number of clearances made by the opposition team



In [486]:
def check_tag(df, tag_id):
    for t in df['tags']:
        if t['id'] == tag_id:
            return True
    return False



def get_stats(df, minute, event_id, subevent_id=-1, tag_id=-1):
    
    def fetch_data(cond, p):
        return df[
            (df['matchPeriod'] == p) &
            (df['eventId'] == event_id) &
            (cond)
        ]

    if minute < 45:
        part = '1H'
        sec = minute*60
    else:
        part = '2H'
        sec = (minute-45)*60
    
    if subevent_id == -1:
        stats_before = fetch_data(df['eventSec'] < sec, part)
        stats_after = fetch_data(df['eventSec'] > sec, part)
    else:
        stats_before = fetch_data((df['eventSec'] < sec) & (df['subEventId'] == subevent_id), part)
        stats_after = fetch_data((df['eventSec'] > sec) & (df['subEventId'] == subevent_id), part)

    
    if part == '2H':
        if subevent_id == -1:
            stats_before = pd.concat([stats_before, fetch_data(1, '1H')], ignore_index=True)
        else:
            stats_before = pd.concat([stats_before, fetch_data(df['subEventId'] == subevent_id, '1H')], ignore_index=True)
    else:
        if subevent_id == -1:
            stats_after = pd.concat([stats_after, fetch_data(1, '2H')], ignore_index=True)
        else:
            stats_after = pd.concat([stats_after, fetch_data(df['subEventId'] == subevent_id, '2H')], ignore_index=True)
        
    
    if tag_id != -1:
        stats_before = stats_before[stats_before.apply(check_tag, tag_id=tag_id, axis=1)]
        stats_after = stats_after[stats_after.apply(check_tag, tag_id=tag_id, axis=1)]
        
    if minute == 90:
        minute -= 0.0001
    
    stats_before_norm = round(len(stats_before) * 90/minute, 2)
    stats_after_norm = round(len(stats_after) * 90/(90-minute), 2)
    
    return [stats_before_norm, stats_after_norm, len(stats_before), len(stats_after)]
    
    
def compute_cols(stats, name, norm=False):
    if norm:
        return {
            f'{name}_before_norm': stats[0],
            #f'{name}_after_norm': stats[1],
            #f'{name}_before': stats[2],
            #f'{name}_after': stats[3]
        }
    return {
            f'{name}_before': stats[2],
            #f'{name}_after': stats[3]
        }

def get_team_name(g_id):
    return teams_df[teams_df['wyId'] == g_id].iloc[0]['name']

def get_player_name(p_id):
    return players_df[players_df['wyId'] == p_id].iloc[0]['shortName']

def get_player_role(p_id):
    return players_df[players_df['wyId'] == p_id].iloc[0]['role']['name']

In [516]:
stats = []

#EVENTS
FOUL_ID = 2
FREE_KICK_ID = 3
OTHERS_ID = 7
PASSES_ID = 8
SAVE_ATTEMPT_ID = 9
SHOTS_ID = 10

#SUBEVENTS
CORNERS = 30
ACCELERATION = 70
CLEARANCE = 71
PASS_CROSS = 80

#TAGS
TAG_GOAL_ID = 101
TAG_OPPORTUNITY = 201
TAG_ASSIST_ID = 301
TAG_KEY_PASS = 302
TAG_YELLOW_CARD = 1702
TAG_RED_CARD = 1701

for ind, row in city_games_df.iterrows():
    
    g_id = row['wyId']
    
    teams_ids = list(row['teamsData'].keys())
    teams_ids.remove(man_city_id)
    other_team_id = teams_ids[0]
    
    home_city = row['venue'] == 'Etihad Stadium'
    
    df_events_game_city = df_events[(df_events['matchId'] == g_id) & (df_events['teamId'] == int(man_city_id))]
    df_events_game_other = df_events[(df_events['matchId'] == g_id) & (df_events['teamId'] == int(other_team_id))]
    
    for s in row['teamsData'][man_city_id]['formation']['substitutions']:
        
        # exclude substitutions nearly the end of the game
        if 88 < s['minute'] or s['minute'] < 45:
            pass

        # NUMBER OF SHOTS
        shots_all_dict = compute_cols(get_stats(df_events_game_city, s['minute'],
                            event_id=SHOTS_ID),
                    'shots', True)
        
        # NUMBER OF SHOTS - GOALS
        shots_goals_dict = compute_cols(get_stats(df_events_game_city, s['minute'],
                            event_id=SHOTS_ID,
                            tag_id=TAG_GOAL_ID),
                    'shots_goals')
        
        # NUMBER OF SHOTS - OPPORTUNITY
        shots_opp_dict = compute_cols(get_stats(df_events_game_city, s['minute'],
                            event_id=SHOTS_ID,
                            tag_id=TAG_OPPORTUNITY),
                    'shots_opp', True)
        
        
        # NUMBER OF PASSES
        passes_all_dict = compute_cols(get_stats(df_events_game_city, s['minute'], 
                               PASSES_ID),
                    'passes_all', True)
        
        
        # NUMBER OF KEY PASSES
        passes_key_dict = compute_cols(get_stats(df_events_game_city, s['minute'], 
                               event_id=PASSES_ID, 
                               tag_id=TAG_KEY_PASS),
                    'passes_keys', True)
        
        # NUMBER OF CROSS PASSES
        passes_cross_dict = compute_cols(get_stats(df_events_game_city, s['minute'], 
                               event_id=PASSES_ID, 
                               subevent_id=PASS_CROSS),
                    'passes_crosses', True)
        
        
        # NUMBER OF PASSES - ASSISTS
        passes_assists_dict = compute_cols(get_stats(df_events_game_city, s['minute'], 
                               event_id=PASSES_ID, 
                               tag_id=TAG_ASSIST_ID),
                    'passes_assists')
        
        # NUMBER OF PASSES - OPPORTUNITY
        passes_opp_dict = compute_cols(get_stats(df_events_game_city, s['minute'],
                            event_id=PASSES_ID,
                            tag_id=TAG_OPPORTUNITY),
                    'passes_opp', True)
        
        # NUMBER OF FREE_KICKS
        freekicks_dict = compute_cols(get_stats(df_events_game_city, s['minute'],
                                event_id=FREE_KICK_ID),
                    'freekicks')

        # NUMBER OF CORNERS
        corners_dict = compute_cols(get_stats(df_events_game_city, s['minute'],
                                event_id=FREE_KICK_ID,
                                subevent_id=CORNERS),
                    'corners')

        # NUMBER OF ACCELERATIONS
        accel_key_dict = compute_cols(get_stats(df_events_game_city, s['minute'], 
                               event_id=OTHERS_ID, 
                               subevent_id=ACCELERATION),
                    'accelerations', True)
        
        
        
        ############ STATS CONCERNING OPPOSITION #############

        # NUMBER OF YELLOW CARDS
        yellow_key_dict = compute_cols(get_stats(df_events_game_other, s['minute'], 
                               event_id=FOUL_ID, 
                               tag_id=TAG_YELLOW_CARD),
                    'yellow_cards')

        # NUMBER OF RED CARDS
        red_key_dict = compute_cols(get_stats(df_events_game_other, s['minute'], 
                               event_id=FOUL_ID, 
                               tag_id=TAG_RED_CARD),
                    'red_cards')
        
        # NUMBER OF SAVE ATTEMPTS
        saves_key_dict = compute_cols(get_stats(df_events_game_other, s['minute'], 
                               event_id=SAVE_ATTEMPT_ID), 
                    'save_attempts', True)
        
        # NUMBER OF CLEARANCES
        clear_key_dict = compute_cols(get_stats(df_events_game_other, s['minute'], 
                               event_id=OTHERS_ID, 
                               subevent_id=CLEARANCE),
                    'clearances', True)
    
        # NUMBER OF SHOTS - GOALS
        shots_goals_against_dict = compute_cols(get_stats(df_events_game_other, s['minute'],
                            event_id=SHOTS_ID,
                            tag_id=TAG_GOAL_ID),
                    'shots_goals_against')
        
        
        sub_stats = {**shots_all_dict, 
                     **shots_goals_dict,
                     **shots_opp_dict,
                     **passes_all_dict,
                     **passes_key_dict,
                     **passes_cross_dict,
                     **passes_assists_dict,
                     **passes_opp_dict,
                     **freekicks_dict,
                     **corners_dict,
                     **accel_key_dict,
                     **yellow_key_dict,
                     **red_key_dict,
                     **saves_key_dict,
                     **clear_key_dict,
                     **shots_goals_against_dict,
                     'min': s['minute'],
                     'other_team': get_team_name(int(other_team_id)),
                     'player_name': get_player_name(s['playerIn']),
                     'player_role': get_player_role(s['playerIn']),
                     'home': home_city
                    }
        
        stats.append(sub_stats)
        
stats_df = pd.DataFrame(stats)
stats_df['diff_goals'] = stats_df['shots_goals_before'] - stats_df['shots_goals_against_before']

In [518]:
stats_df.head()

,shots_before_norm,shots_goals_before,shots_opp_before_norm,passes_all_before_norm,passes_keys_before_norm,passes_crosses_before_norm,passes_assists_before,passes_opp_before_norm,freekicks_before,corners_before,...,red_cards_before,save_attempts_before_norm,clearances_before_norm,shots_goals_against_before,min,other_team,player_name,player_role,home,diff_goals
0,9.15,0,4.58,675.76,1.53,22.88,0,0.0,32,5,...,0,0.00,15.25,0,59,Southampton,Gabriel Jesus,Forward,False,0
1,11.54,0,6.92,600.00,1.15,23.08,0,0.0,41,9,...,0,1.15,18.46,0,78,Southampton,Brahim Diaz,Midfielder,False,0
2,10.98,0,6.59,574.02,1.10,21.95,0,0.0,43,9,...,0,1.10,17.56,0,82,Southampton,P. Foden,Midfielder,False,0
3,18.95,3,14.21,819.47,4.74,18.95,3,0.0,30,4,...,0,7.11,22.50,1,76,Brighton & Hove Albion,B. Mendy,Defender,True,2
4,18.43,3,14.10,808.92,5.42,17.35,3,0.0,32,4,...,0,7.59,20.60,1,83,Brighton & Hove Albion,Brahim Diaz,Midfielder,True,2


## 2. Final rating of the player that entered in the game

This was done by scrapping the players ratings from whoscored site.
This information was extracted and used purely on a personal level.

In [17]:
CHROME_DRIVER_PATH = "chromedriver_path"

def load_html(url):
    driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
    driver.get(url)
    #time.sleep(8)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit()
    
    return soup


def get_team_subs(soup):
    def get_subs(home_away):
        
        team = soup.find('span', {'class': f'col12-lg-4 col12-m-4 col12-s-0 col12-xs-0 {home_away} team'}).text

        home_stats = soup.find('div', {'id': f'statistics-table-{home_away}-summary'})
        players_stats = home_stats.findAll('tr')

        for p in players_stats:
            if len(p.find_all('span', {'data-type': "19"})) > 0:
                name = p.find('span', {'class': "iconize iconize-icon-left"}).text
                rating = p.find('td', {'class': "rating"}).text

                yield {'home': home_away == 'home',
                       'rating': rating,
                       'team': team,
                       'player': name
                      }
    return get_subs

In [ ]:
# fetch all games
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')

url = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/6829/Stages/15151/Fixtures/England-Premier-League-2017-2018'
driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
driver.get(url)
time.sleep(7)

p_max = 10

subs_df = []
links = []

for p in range(p_max):
    print('next')
    time.sleep(5)
    
    if p != 0:
        target = driver.find_element_by_xpath("//*[@class='previous button ui-state-default rc-l is-default']")
        target.click()
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    m_reports = soup.find_all('a', {'class': 'match-link match-report rc'})
    
    i_counter = 0
    
        
    # iterate each game and extract subs and respective scores
    while i_counter != len(m_reports)-1:
        m = m_reports[i_counter]
        g = m['href']
        
        complete_url = f'https://www.whoscored.com/{g}'.replace('MatchReport', 'LiveStatistics')
        links.append(complete_url)
        
        print("Acessing: ", complete_url)

        m_driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
        m_driver.get(complete_url)
        
        time.sleep(10)
        
        m_soup = BeautifulSoup(m_driver.page_source, 'html.parser')

        match = get_team_subs(m_soup)
        
        stats_home = list(match('home'))
        stats_away = list(match('away'))

        
        print(stats_home)
        print(stats_away)
        
        home_team = m_soup.find('span', {'class': f'col12-lg-4 col12-m-4 col12-s-0 col12-xs-0 home team'}).text
        away_team = m_soup.find('span', {'class': f'col12-lg-4 col12-m-4 col12-s-0 col12-xs-0 away team'}).text
        
        for i in range(len(stats_home)):
            stats_home[i]['other_team'] = away_team
        
        for i in range(len(stats_away)):
            stats_away[i]['other_team'] = home_team

        
        if len(stats_home) + len(stats_home) > 0:
            i_counter += 1
        else:
            time.sleep(3)
    
        subs_df.append(stats_home)
        subs_df.append(stats_away)
        
        i_counter += 1
        #m_driver.quit()



## Join two data sources

This is where I joined the stats data with the player ratings

In [528]:
complete_team_names = list(stats_df['other_team'].unique())

def parse_strange_char(word):
    new_word = word.replace('\\u011f', 'ğ').replace('\\u011e', 'Ğ').replace('\\u0131', 'ı').replace('\\u0130', 'İ')\
            .replace('\\u00f6', 'ö').replace('\\u00d6', 'Ö').replace('\\u00fc', 'ü').replace('\\u00dc', 'Ü')\
            .replace('\u015f', 'ş').replace('\\u015e', 'Ş').replace('\\u00e7', 'ç').replace('\\u00c7', 'Ç')\
            .replace('\\u00e9', 'é').replace('\\u00e1', 'á').replace('\\u00f1', 'ñ')

    new_word = new_word.split(' ')[0][0] + '. ' + new_word.split(' ')[-1]
    
    #remove accents
    return unidecode.unidecode(new_word)



stats_df['player_name'] = stats_df['player_name'].apply(parse_strange_char)
ratings_df['player'] = ratings_df['player'].apply(parse_strange_char)


complete_player_names = list(stats_df['player_name'].unique())



def sync_team_names(name, teams_names):
    
    for t in teams_names:
        if name in t:
            return t
    return name


ratings_df['other_team_matched'] = ratings_df['other_team'].apply(sync_team_names, args=(complete_team_names,))


def sync_player_names(name, teams_names):
    for t in teams_names:
        if name in t:
            return t
    print('NOT FOUND', name)
    return name


ratings_df['player_matched'] = ratings_df['player'].apply(sync_player_names, args=(complete_player_names,))
stats_df['player_name'] = stats_df['player_name'].apply(parse_strange_char)

In [ ]:
#join two dataframes

def find_row(row, df):
    matched_rows = df[(df['other_team_matched'] == row['other_team']) & (df['player_matched'] == row['player_name'])\
                     & (df['home'] == row['home'])]
    
    #display(matched_rows)
    assert len(matched_rows) == 1, 'The two dataframes are inconcistent'
    
    return matched_rows.iloc[0]['rating']


stats_df['rating'] = stats_df.apply(find_row, args=(ratings_df, ), axis=1)

The final *stats_df*, which comprise the stats data and also the players rating, will be used as input in the *model_development* notebook